In [5]:
import pandas as pd

# -------------------------
# CONFIGURATION & CONSTANTS
# -------------------------
TURKISH_MONTHS = {
    'Ocak': 1, 'Şubat': 2, 'Mart': 3, 'Nisan': 4, 'Mayıs': 5, 'Haziran': 6,
    'Temmuz': 7, 'Ağustos': 8, 'Eylül': 9, 'Ekim': 10, 'Kasım': 11, 'Aralık': 12
}

# -------------------------
# FUNCTIONS
# -------------------------

def load_index_data(filepath):
    df = pd.read_excel(filepath)
    df.columns = df.iloc[3][:3].to_list() + df.iloc[1][3:].to_list()
    df = df.iloc[4:]
    df = df.drop(columns=df.columns[df.columns.isna()])
    df = df.melt(id_vars=["Year", "Months"], var_name="inflation_code", value_name="index")
    df = df[(df['inflation_code'].str.len() == 5) | (df['inflation_code'].str.len() == 1)]
    return df

def load_weights(filepath):
    return pd.read_excel(filepath, dtype={"inflation_code": str})

def preprocess_data(df, weights):
    df = df.merge(weights, on=["inflation_code", "Year"], how="left")
    df = df[df.Year > 2014].fillna(100)
    df['Month_Num'] = df['Months'].map(TURKISH_MONTHS)
    df.sort_values(by=['inflation_code', 'Year', 'Month_Num'], inplace=True)
    return df

def calculate_cumulative_inflation(df):
    dec_df = df[df['Month_Num'] == 12][['Year', 'inflation_code', 'index']].copy()
    dec_df['Year'] += 1
    dec_df.rename(columns={'index': 'Prev_December_Index'}, inplace=True)
    df = df.merge(dec_df, on=['inflation_code', 'Year'], how='left')
    df['Cumulative_Inflation_%'] = ((df['index'] / df['Prev_December_Index']) - 1) * 100
    return df.dropna(subset=['Cumulative_Inflation_%'])

def compute_contributions(df):
    df['total_contribution'] = df['Cumulative_Inflation_%'] * df['weights'] / 100
    df['monthly_inf'] = df['index'].pct_change() * 100
    df.sort_values(by=['inflation_code', 'Year', 'Month_Num'], inplace=True)
    df['monthly_contribution_diff'] = df.groupby('inflation_code')['total_contribution'].diff()
    return df

def compute_correction(df):
    main_inf = df[df['inflation_code'] == '0'][['Year', 'Month_Num', 'monthly_inf']].rename(
        columns={'monthly_inf': 'main_monthly_inf'}
    )

    subgroup_df = df[df['inflation_code'] != '0']
    subgroup_sum = subgroup_df.groupby(['Year', 'Month_Num'])['monthly_contribution_diff'].sum().reset_index()
    subgroup_sum.rename(columns={'monthly_contribution_diff': 'total_contrib_diff'}, inplace=True)

    correction = pd.merge(main_inf, subgroup_sum, on=['Year', 'Month_Num'])
    correction['correction_coef'] = correction['main_monthly_inf'] / correction['total_contrib_diff']
    return correction[['Year', 'Month_Num', 'correction_coef']]

def apply_correction(df, correction):
    df = df.merge(correction, on=["Year", "Month_Num"])
    df['monthly_contribution'] = df['monthly_contribution_diff'] * df['correction_coef']
    return df

# -------------------------
# MAIN EXECUTION PIPELINE
# -------------------------
def process_inflation_data(index_path, weights_path, output_path="monthly_contribution.xlsx"):
    df = load_index_data(index_path)
    weights = load_weights(weights_path)
    df = preprocess_data(df, weights)
    df = calculate_cumulative_inflation(df)
    df = compute_contributions(df)
    correction = compute_correction(df)
    df = apply_correction(df, correction)

    # Select and sort final output
    output_columns = [
        'Year', 'Months', 'Month_Num', 'inflation_code', 'index',
        'Prev_December_Index', 'Cumulative_Inflation_%', 'weights',
        'total_contribution', 'monthly_inf', 'monthly_contribution_diff',
        'correction_coef', 'monthly_contribution'
    ]
    df = df[output_columns].dropna()
    df.sort_values(by=['Year', 'Month_Num', 'inflation_code'], inplace=True)
    
    df.to_excel(output_path, index=False)
    return df

# -------------------------
# RUN THE PIPELINE
# -------------------------
process_inflation_data(
        index_path="raw_data/harcama gruplarina gore endeks sonuclari.xlsx",
        weights_path="raw_data/cpi_item_weights_subgroups_5.xlsx",
        output_path="monthly_contribution.xlsx"
    )



/var/folders/lx/dlxz3kpj1pq9fx9tbtzznnr00000gn/T/ipykernel_2424/2209028270.py:29: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df[df.Year > 2014].fillna(100)


,Year,Months,Month_Num,inflation_code,index,Prev_December_Index,Cumulative_Inflation_%,weights,total_contribution,monthly_inf,monthly_contribution_diff,correction_coef,monthly_contribution
1,2016,Şubat,2,0,274.38,269.54,1.795652,100.000000,1.795652,-0.021863,-0.022260,-0.000294,0.000007
117,2016,Şubat,2,01111,329.89,333.51,-1.085425,100.000000,-1.085425,-0.626563,-0.623669,-0.000294,0.000183
233,2016,Şubat,2,01112,279.31,274.13,1.889614,100.000000,1.889614,1.254305,1.262175,-0.000294,-0.000371
349,2016,Şubat,2,01113,336.28,319.99,5.090784,100.000000,5.090784,2.793911,2.856339,-0.000294,-0.000839
465,2016,Şubat,2,01114,273.67,268.64,1.872394,100.000000,1.872394,1.085953,1.094401,-0.000294,-0.000322
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16239,2025,Ağustos,8,12520,2829.15,2235.43,26.559543,0.019035,0.005056,2.228750,0.000525,0.839797,0.000441
16355,2025,Ağustos,8,12530,2772.31,2174.36,27.500046,0.033056,0.009090,1.151879,0.000480,0.839797,0.000403
16471,2025,Ağustos,8,12540,5298.63,4345.17,21.942985,0.201346,0.044181,0.245002,0.000600,0.839797,0.000504
16587,2025,Ağustos,8,12620,321.98,223.11,44.314464,0.004921,0.002181,0.000000,0.000000,0.839797,0.000000
